# 股票预测模型工作流

---
### 工作流说明
1.  **阶段零 (Setup)**: 导入库、加载配置。
2.  **阶段一 (Data Pipeline)**: 独立运行。负责处理并保存数据，生成 L2 特征数据缓存。
3.  **阶段二 (Model Pipeline)**: 独立运行。包含三个子步骤：
    - **2.1 HPO**: 自动调参。
    - **2.2 (预处理)**: 智能地加载或生成 L3 预处理数据缓存
    - **2.3 (模型训练)**: 使用 L3 缓存进行高效的模型训练。
    - **2.4 (评估)**: 对训练结果进行聚合与可视化。

## 0. 通用设置与导入

In [1]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# 设置 Matplotlib 样式 (可以在 main_train 外部设置)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# --- 动态导入 main_train ---
project_root = str(Path().resolve())
if project_root not in sys.path:
    sys.path.append(project_root)

# 从我们统一的核心引擎中导入所有需要的函数和模块
from main_train import (
    run_load_config_and_modules,
    run_all_data_pipeline,
    run_preprocess_l3_cache,
    run_hpo_train,
    run_all_models_train,
    run_performance_evaluation,
    run_results_visualization
)

# --- 加载全局配置和模块 ---
# 在 Notebook 的生命周期中，我们只加载一次
config, modules = run_load_config_and_modules()

if not (config and modules):
    raise RuntimeError("环境初始化失败，请检查配置文件路径和模块导入。")

d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\main_train.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


--- 正在初始化环境：加载配置与模块... ---
INFO: 底层并行计算库线程数已设置为: 4
INFO: 项目模块导入成功。
SUCCESS: 配置已从 'configs/config.yaml' 加载。


# **阶段一：数据准备与特征工程**

In [2]:
USE_LATEST_DATE = True
if config and modules:
    print(f"--- INFO: 日期模式 -> {'使用最新日期' if USE_LATEST_DATE else '使用配置文件中的固定日期'} ---")
    
    # (核心修改) 将开关状态传递给核心引擎函数
    run_all_data_pipeline(config, modules, use_today_as_end_date=USE_LATEST_DATE)

--- INFO: 日期模式 -> 使用最新日期 ---
=== 阶段一：数据准备与特征工程 ===
INFO: 已启用动态日期模式，将使用今天的日期 '2025-10-24' 作为 end_date。
      计算得出的 start_date 为: 2010-10-24
INFO: 正在尝试登录 Baostock...
login success!
INFO: Baostock API 登录成功。
INFO: 未在配置中提供有效的 Tushare Token，将跳过 Tushare 相关数据。
开始执行数据管道协调任务...
INFO: 正在根据 'end_date' 和 'data_lookback_years' 动态计算 'start_date'...
      计算得出的 start_date 为: 2010-10-24，已更新到本次运行的全局配置中。
INFO: 特征文件已存在于 data\processed\000100.SZ\5de9c41e74af\features.pkl，跳过 TCL科技 (000100.SZ) 的数据处理。
INFO: 特征文件已存在于 data\processed\000426.SZ\f6589d437087\features.pkl，跳过 兴业矿业 (000426.SZ) 的数据处理。
INFO: 特征文件已存在于 data\processed\002083.SZ\e54a78b13e0b\features.pkl，跳过 孚日股份 (002083.SZ) 的数据处理。
INFO: 特征文件已存在于 data\processed\000150.SZ\78537e2c1be4\features.pkl，跳过 宜华健康 (000150.SZ) 的数据处理。
INFO: 特征文件已存在于 data\processed\300013.SZ\681eb130ef3c\features.pkl，跳过 新宁物流 (300013.SZ) 的数据处理。
INFO: 特征文件已存在于 data\processed\300242.SZ\f49537315712\features.pkl，跳过 佳云科技 (300242.SZ) 的数据处理。
INFO: 特征文件已存在于 data\processed\002006.SZ\8e20872224e0

# **阶段二：模型训练与评估**

### 2.1 数据预加载与全局预处理 (L3 缓存)

In [ ]:
if config and modules:
    # 调用核心引擎中的预处理函数
    # force_reprocess=True/False 可以方便地控制是否重建缓存
    global_data_cache = run_preprocess_l3_cache(config, modules, force_reprocess=False)


=== 工作流阶段 2.1：为模型预处理数据 (L3 缓存) ===
WARNNING: 加载 L3 缓存失败: 。将重新预处理。
INFO: L3 缓存不存在、为空或被强制重建。开始执行预处理流程...



正在预处理股票:   0%|          | 0/8 [00:00<?, ?it/s]

### 2.2 超参数优化

In [ ]:
# 同样，可以通过一个简单的开关来控制是否运行
RUN_HPO = False

if RUN_HPO and config and modules and 'global_data_cache' in locals():
    # 调用核心引擎中的 HPO 函数
    # HPO 的结果会自动更新到内存中的 config 字典里
    run_hpo_train(config, modules, global_data_cache)

### 2.3 模型训练

In [ ]:
if config and modules and 'global_data_cache' in locals():
    # 调用核心引擎中的模型训练函数
    # 同样可以通过开关控制是否强制重训
    all_ic_history = run_all_models_train(
        config, 
        modules, 
        global_data_cache, 
        force_retrain_base=True, 
        force_retrain_fuser=True
    )

=== 工作流阶段 2.3：训练所有模型 ===

--- 2.3.1 基础模型训练 ---


训练基础模型:   0%|          | 0/8 [00:00<?, ?it/s]

--- 开始为 TCL科技 (000100.SZ) 进行 LGBM 模型训练 ---
INFO: 强制重训已开启，将删除所有旧的构件...
INFO: 开始对 TCL科技 (000100.SZ) 进行跨 64 folds 的前向验证...


正在 TCL科技 上训练 LGBM:   0%|          | 0/64 [00:00<?, ?it/s]

INFO: 滚动训练成功完成，准备训练最终模型。
INFO: 正在训练最终模型...
SUCCESS: 新版本 (20251024) 模型已保存: lgbm_model_20251024.pkl
INFO: 整个训练流程（包括最终模型）成功完成，已移除进度文件。
--- 开始为 TCL科技 (000100.SZ) 进行 LSTM 模型训练 ---
INFO: 强制重训已开启，将删除所有旧的构件...
INFO: PyTorch LSTMBuilder initialized with device: CUDA
INFO: DataLoader will use 8 parallel workers.
INFO: Automatic Mixed Precision (AMP) is ENABLED (float16).
INFO: 开始对 TCL科技 (000100.SZ) 进行跨 64 folds 的前向验证...


正在 TCL科技 上训练 LSTM:   0%|          | 0/64 [00:00<?, ?it/s]

    - Epochs (LSTM):   0%|          | 0/75 [00:00<?, ?it/s]

RuntimeError: DataLoader worker (pid(s) 26180, 29652, 27456, 29568, 31360, 27576, 27680, 7836) exited unexpectedly

### 2.4 结果聚合、评估与可视化

In [ ]:
if config and modules and 'all_ic_history' in locals() and all_ic_history:
    # 调用核心引擎中的评估和可视化函数
    evaluation_summary, backtest_summary, final_eval_df = run_performance_evaluation(config, modules, all_ic_history)
    
    # 只有在有结果时才进行可视化
    if evaluation_summary is not None or backtest_summary is not None:
        run_results_visualization(config, modules, evaluation_summary, backtest_summary, final_eval_df)
    else: print('无可视化结果')